## Scrapping data from the given link

In [39]:
from selenium.common.exceptions import NoSuchElementException, TimeoutException

In [15]:
from selenium.webdriver.common.by import By
import awscli
import boto3
import selenium
import pandas as pd
import time
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager


####SCRAPE THE WEBSITE######
###call the webdriver
s=Service(ChromeDriverManager().install())
browser = webdriver.Chrome(service=s)

#enter the url path that needs to be accessed by webdriver
browser.get('https://www.charitiesnys.com/RegistrySearch/search_charities.jsp')

#identify xpath of location to select element
inputElement = browser.find_element(By.XPATH, "/html/body/div/div[2]/div/table/tbody/tr/td[2]/div/div/font/font/font/font/font/font/table/tbody/tr[4]/td/form/table/tbody/tr[2]/td[2]/input[1]")
inputElement.send_keys('0')
inputElement1 = browser.find_element(By.XPATH,"/html/body/div/div[2]/div/table/tbody/tr/td[2]/div/div/font/font/font/font/font/font/table/tbody/tr[4]/td/form/table/tbody/tr[10]/td/input[1]").click()

#identify the table to scrape
table = browser.find_element(By.CSS_SELECTOR,'table.Bordered')
sleep(1)

#####CREATE DATE FRAME#####
#create empty dataframe
df =[]

#loop through dataframe to export table
for row in table.find_elements(By.CSS_SELECTOR,'tr'):
    if row.find_elements(By.XPATH, './parent::thead'):
        continue  # Skip rows within the <thead> section
    cols = df.append([cell.text for cell in row.find_elements(By.CSS_SELECTOR,'td')])

df
#update dataframe with header 
df = pd.DataFrame(df, columns = ["Organization Name", "NY Reg #", "EIN" ,"Registrant Type","City","State"])
display(df) #let's have a look at the data before creating the CSV file and loading it into s3 



,Organization Name,NY Reg #,EIN,Registrant Type,City,State
0,None,None,None,None,None,None
1,"""Forever Captain Poodaman"" The Ahmad Butler Fo...",48-07-16,843800926,NFP,PHILADELPHIA,PA
2,"""Incredibly Blessed"" Inc",49-54-61,842071758,NFP,STATEN ISLAND,NY
3,"""R"" S.U.C.C.E.S.S. Foundation Inc.",49-06-59,874012670,NFP,ROCHESTER,NY
4,"""Studio 5404"" Inc.",44-39-58,463180470,NFP,MASSAPAQUA,NY
5,"""THEY ARE HAITIAN"" FUND, INC.",20-63-46,300170128,NFP,HUDSON,NY
6,"""Y"" Dive, Inc.",48-45-01,854252095,NFP,SAINT ALBANS,NY
7,(ASMA) American Syrian Multicultural Associati...,42-84-63,273130182,NFP,BROOKLYN,NY
8,#FeedHamburg,48-37-35,854150318,NFP,HAMBURG,NY
9,#HicksStrong Inc.,48-10-48,842612081,NFP,CLIFTON PARK,NY


## Importing data into S3 bucket

In [18]:

###LOAD THE FILE INTO S3####
# prepare csv file name   
pathname = 's3://mybucketmodule10/' #specify location of s3:/{my-bucket}/
filename= 'charities_bureau_scrape_' #name of your group
datetime = time.strftime("%Y%m%d%H%M%S") #timestamp
filenames3 = "%s%s%s.csv"%(pathname,filename,datetime) #name of the filepath and csv file

#load file into s3. Pandas actually leverages boto to connect to s3 and can push the file directly into an s3 bucket
df.to_csv(filenames3, header=True) 

#print success message
print("Successfull uploaded file to location:"+str(filenames3))

Successfull uploaded file to location:s3://mybucketmodule10/charities_bureau_scrape_20240413170021.csv


## Evidence that my file is updated in the my bucket

In [41]:
# Generic function to read and validate the data loaded into S3 bucket
def print_bucket_contents(bucket):
    s3 = boto3.client('s3')
    response = s3.list_objects_v2(Bucket=bucket)

    if 'Contents' in response:
        print("Contents of the '{}' bucket:".format(bucket))
        for obj in response['Contents']:
            print(obj['Key'])
    else:
        print("The bucket '{}' is empty.".format(bucket))

In [42]:
print_bucket_contents('mybucketmodule10')

Contents of the 'mybucketmodule10' bucket:
charities_bureau_scrape_20240413162806.csv
charities_bureau_scrape_with_one_row_blank20240413162806.csv


## Part 2 - Data From All Pages

In [45]:

####SCRAPE THE WEBSITE######
###call the webdriver
s=Service(ChromeDriverManager().install())
browser = webdriver.Chrome(service=s)

#enter the url path that needs to be accessed by webdriver
browser.get('https://www.charitiesnys.com/RegistrySearch/search_charities.jsp')

#identify xpath of location to select element
inputElement = browser.find_element(By.XPATH, "/html/body/div/div[2]/div/table/tbody/tr/td[2]/div/div/font/font/font/font/font/font/table/tbody/tr[4]/td/form/table/tbody/tr[2]/td[2]/input[1]")
inputElement.send_keys('0')
inputElement1 = browser.find_element(By.XPATH,"/html/body/div/div[2]/div/table/tbody/tr/td[2]/div/div/font/font/font/font/font/font/table/tbody/tr[4]/td/form/table/tbody/tr[10]/td/input[1]").click()

df = []  # Empty list to store data from each page
while True:
    try:
        element_next = browser.find_element(By.XPATH, '//span[@class="pagelinks"]/a[text()="Next"]') #finding element which is having link of next page
        sleep(1)
        table = browser.find_element(By.CSS_SELECTOR,'table.Bordered')
        sleep(1)
        for row in table.find_elements(By.CSS_SELECTOR,'tr'):
            if row.find_elements(By.XPATH, './parent::thead'):
                continue  # Skip rows within the <thead> section
            df.append([cell.text for cell in row.find_elements(By.CSS_SELECTOR,'td')])
        element_next.click()
        sleep(4)
    except NoSuchElementException:
        table = browser.find_element(By.CSS_SELECTOR,'table.Bordered')
        sleep(1)
        for row in table.find_elements(By.CSS_SELECTOR,'tr'):
            if row.find_elements(By.XPATH, './parent::thead'):
                continue  # Skip rows within the <thead> section
            df.append([cell.text for cell in row.find_elements(By.CSS_SELECTOR,'td')])
        # Handle the case where "Next" button is not found (reached last page)
        print("Data Added from all the pages.")
        break
#update dataframe with header 
df = pd.DataFrame(df, columns = ["Organization Name", "NY Reg #", "EIN" ,"Registrant Type","City","State"])
display(df) #let's have a look at the data before creating the CSV file and loading it into s3 

Data Added from all the pages.


,Organization Name,NY Reg #,EIN,Registrant Type,City,State
0,"""Forever Captain Poodaman"" The Ahmad Butler Fo...",48-07-16,843800926,NFP,PHILADELPHIA,PA
1,"""Incredibly Blessed"" Inc",49-54-61,842071758,NFP,STATEN ISLAND,NY
2,"""R"" S.U.C.C.E.S.S. Foundation Inc.",49-06-59,874012670,NFP,ROCHESTER,NY
3,"""Studio 5404"" Inc.",44-39-58,463180470,NFP,MASSAPAQUA,NY
4,"""THEY ARE HAITIAN"" FUND, INC.",20-63-46,300170128,NFP,HUDSON,NY
...,...,...,...,...,...,...
95,University of Virginia Health Foundtion,40-44-88,412097394,NFP,CHARLOTTESVILLE,VA
96,Violin Player,41-40-19,270773158,NFP,LONDONDERRY,NH
97,"William A. Epps Community Center, Inc.",40-91-11,861074714,NFP,STATEN ISLAND,NY
98,WORLD SOCIETY OF CZESTOCHOWA JEWS AND THEIR DE...,40-46-49,205101779,NFP,NEW YORK,NY


In [46]:

###LOAD THE FILE INTO S3####
# prepare csv file name   
pathname = 's3://mybucketmodule10/' #specify location of s3:/{my-bucket}/
filename= 'charities_bureau_scrape_allpage_' #name of your group
datetime = time.strftime("%Y%m%d%H%M%S") #timestamp
filenames3 = "%s%s%s.csv"%(pathname,filename,datetime) #name of the filepath and csv file

#load file into s3. Pandas actually leverages boto to connect to s3 and can push the file directly into an s3 bucket
df.to_csv(filenames3, header=True) 

#print success message
print("Successfull uploaded file to location:"+str(filenames3))

Successfull uploaded file to location:s3://mybucketmodule10/charities_bureau_scrape_allpage_20240414152605.csv


In [48]:
print_bucket_contents('mybucketmodule10')

Contents of the 'mybucketmodule10' bucket:
charities_bureau_scrape_20240413162806.csv
charities_bureau_scrape_allpage_20240414152605.csv
charities_bureau_scrape_with_one_row_blank20240413162806.csv
